In [1]:
import pandas as pd
import re
from haversine import haversine

In [2]:
df = pd.read_csv('Dataset/airbnb.csv')
df.city.unique()

array(['Chicago', 'NYC', 'LA', 'SF', 'DC', 'Boston'], dtype=object)

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_wikipedia_table(url):
    response = requests.get(url)
    if response.status_code != 200:
        print(f'Failed to retrieve the page, status code: {response.status_code}')
        return None

    soup = BeautifulSoup(response.content, 'html.parser')
    tables = soup.find_all('table', {'class': 'wikitable'})

    if len(tables) == 0:
        print('No "wikitable" found on this page.')
        return None
    table = tables[0]
    headers = []
    for th in table.find_all('th'):
        headers.append(th.get_text(strip=True))
    rows = []
    for tr in table.find_all('tr')[1:]:
        cols = tr.find_all('td')
        row = [col.get_text(strip=True) for col in cols]
        if row: 
            rows.append(row)
    df = pd.DataFrame(rows, columns=headers)
    return df


In [4]:
regex = r'(\d{1,3}°\d{1,2}′\d{1,2}″[NS])\s+(\d{1,3}°\d{1,2}′\d{1,2}″[EW])'


In [5]:
boston = pd.read_csv('Dataset/landmarks/Boston.csv')
boston[['latitude', 'longitude']] = boston['Location'].str.extract(regex)
boston = boston.dropna(subset=['latitude', 'longitude'])
print(boston[['latitude', 'longitude']].head())

     latitude   longitude
0  42°21′36″N  71°03′56″W
1  42°21′25″N  71°03′59″W
2  42°17′52″N  71°07′22″W
3  42°20′57″N  71°05′25″W
4  42°21′30″N  71°03′58″W


In [6]:
chicago = pd.read_csv('Dataset/landmarks/Chicago.csv')
chicago[['latitude', 'longitude']] = chicago['Location'].str.extract(regex)
chicago = chicago.dropna(subset=['latitude', 'longitude'])
print(chicago[['latitude', 'longitude']].head())

     latitude   longitude
1  41°52′47″N  87°38′09″W
2  41°53′16″N  87°37′27″W
3  41°53′11″N  87°37′36″W
4  41°56′51″N  87°39′00″W
5  41°46′53″N  87°34′26″W


In [7]:
DC = pd.read_csv('Dataset/landmarks/DC.csv')
DC[['latitude', 'longitude']] = DC['Location'].str.extract(regex)
DC = DC.dropna(subset=['latitude', 'longitude'])
print(DC[['latitude', 'longitude']].head())

     latitude   longitude
0  38°54′05″N  77°02′46″W
1  38°54′39″N  77°02′07″W
2  38°54′12″N  77°01′28″W
3  38°53′58″N  77°02′19″W
4  38°54′39″N  77°02′53″W


In [8]:
LA = pd.read_csv('Dataset/landmarks/LA.csv')
LA[['latitude', 'longitude']] = LA['Location'].str.extract(regex)
LA = LA.dropna(subset=['latitude', 'longitude'])
print(LA[['latitude', 'longitude']].head())

     latitude    longitude
0  34°02′04″N  118°40′51″W
1  34°05′26″N  117°44′35″W
2  34°16′11″N  118°10′06″W
3  34°03′26″N  118°14′16″W
4  33°47′23″N  118°15′32″W


In [9]:
NYC = pd.read_csv('Dataset/landmarks/NYC.csv')
NYC[['latitude', 'longitude']] = NYC['Location'].str.extract(regex)
NYC = NYC.dropna(subset=['latitude', 'longitude'])
print(NYC[['latitude', 'longitude']].head())

     latitude   longitude
0  40°44′30″N  73°59′01″W
1  40°53′32″N  73°51′57″W
2  40°42′52″N  74°00′16″W
3  40°42′17″N  74°00′09″W
4  40°42′32″N  74°00′45″W


In [10]:
SF = pd.read_csv('Dataset/landmarks/SF.csv')
SF[['latitude', 'longitude']] = SF['Description'].str.extract(regex)
SF = SF.dropna(subset=['latitude', 'longitude'])
print(SF[['latitude', 'longitude']].head())

     latitude    longitude
0  37°45′51″N  122°25′36″W
1  37°47′34″N  122°24′21″W
2  37°47′36″N  122°24′06″W
3  37°47′08″N  122°24′13″W
4  37°47′56″N  122°24′28″W


In [11]:
def dms_to_decimal(dms):
    match = re.match(r'(\d+)°(\d+)′(\d+)″([NSWE])', dms)
    
    if match:
        degrees, minutes, seconds, direction = match.groups()
        decimal = int(degrees) + int(minutes)/60 + int(seconds)/3600
        if direction in ['S', 'W']:
            decimal = -decimal
        return decimal
    else:
        raise ValueError(f'Invalid DMS format: {dms}')

In [12]:
boston['latitude_decimal'] = boston['latitude'].apply(dms_to_decimal)
boston['longitude_decimal'] = boston['longitude'].apply(dms_to_decimal)

chicago['latitude_decimal'] = chicago['latitude'].apply(dms_to_decimal)
chicago['longitude_decimal'] = chicago['longitude'].apply(dms_to_decimal)

DC['latitude_decimal'] = DC['latitude'].apply(dms_to_decimal)
DC['longitude_decimal'] = DC['longitude'].apply(dms_to_decimal)

LA['latitude_decimal'] = LA['latitude'].apply(dms_to_decimal)
LA['longitude_decimal'] = LA['longitude'].apply(dms_to_decimal)

LA['latitude_decimal'] = LA['latitude'].apply(dms_to_decimal)
LA['longitude_decimal'] = LA['longitude'].apply(dms_to_decimal)

NYC['latitude_decimal'] = NYC['latitude'].apply(dms_to_decimal)
NYC['longitude_decimal'] = NYC['longitude'].apply(dms_to_decimal)

SF['latitude_decimal'] = SF['latitude'].apply(dms_to_decimal)
SF['longitude_decimal'] = SF['longitude'].apply(dms_to_decimal)

In [13]:
def average_distance_from_landmark(lat, lon, landmark_df: pd.DataFrame):
    distances = landmark_df.apply(
        lambda row: haversine((row['latitude_decimal'], row['longitude_decimal']), (lat, lon)),
        axis=1
    )
    return distances.mean()

In [14]:
df.loc[df['city'] == 'Boston', 'average_distance_from_landmark'] = df[df['city'] == 'Boston'].apply(
    lambda row: average_distance_from_landmark(row['latitude'], row['longitude'], boston), axis=1
)
df.loc[df['city'] == 'Chicago', 'average_distance_from_landmark'] = df[df['city'] == 'Chicago'].apply(
    lambda row: average_distance_from_landmark(row['latitude'], row['longitude'], chicago), axis=1
)
df.loc[df['city'] == 'DC', 'average_distance_from_landmark'] = df[df['city'] == 'DC'].apply(
    lambda row: average_distance_from_landmark(row['latitude'], row['longitude'], DC), axis=1
)
df.loc[df['city'] == 'LA', 'average_distance_from_landmark'] = df[df['city'] == 'LA'].apply(
    lambda row: average_distance_from_landmark(row['latitude'], row['longitude'], LA), axis=1
)
df.loc[df['city'] == 'NYC', 'average_distance_from_landmark'] = df[df['city'] == 'NYC'].apply(
    lambda row: average_distance_from_landmark(row['latitude'], row['longitude'], NYC), axis=1
)
df.loc[df['city'] == 'SF', 'average_distance_from_landmark'] = df[df['city'] == 'SF'].apply(
    lambda row: average_distance_from_landmark(row['latitude'], row['longitude'], SF), axis=1
)

In [15]:
df

,id,property_type,room_type,amenities,accommodates,bathrooms,bed_type,cancellation_policy,cleaning_fee,city,...,name,neighbourhood,number_of_reviews,review_scores_rating,thumbnail_url,zipcode,bedrooms,beds,price,average_distance_from_landmark
0,6242629,Apartment,Private room,"{TV,""Cable TV"",""Wireless Internet"",""Air condit...",2,2.0,Real Bed,flexible,False,Chicago,...,Beautiful apartment in tri-taylor w/private ba...,Little Italy/UIC,12,100,https://a0.muscache.com/im/pictures/7b8e236e-2...,60612,1,1,53.0,7.714067
1,18688272,Apartment,Private room,"{Internet,""Wireless Internet"",""Air conditionin...",2,1.0,Real Bed,strict,True,NYC,...,Quintessential W'burg Apartment,Williamsburg,119,92,https://a0.muscache.com/im/pictures/33099113/4...,11211,1,1,103.0,7.787003
2,15435501,Apartment,Private room,"{TV,Internet,""Wireless Internet"",""Air conditio...",2,1.0,Real Bed,moderate,True,LA,...,Private Room in Hollywood Oasis,Hollywood,47,94,https://a0.muscache.com/im/pictures/978947c3-3...,90028,1,1,58.0,25.945593
3,15822779,House,Private room,"{""Wireless Internet"",Breakfast,""Pets live on t...",2,1.5,Real Bed,moderate,True,SF,...,Bright Bernal room with garden,Bernal Heights,49,94,https://a0.muscache.com/im/pictures/49742373/3...,94110,1,1,115.0,5.173197
4,10752305,Apartment,Entire home/apt,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",5,1.0,Real Bed,strict,True,NYC,...,Modern/Spacious 1 Bedroom Next To Times Square,Hell's Kitchen,23,98,https://a0.muscache.com/im/pictures/e3b8a0a7-f...,10019,1,4,179.0,6.827768
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,18122161,House,Private room,"{""Wireless Internet"",""Air conditioning"",Kitche...",3,1.0,Real Bed,moderate,True,NYC,...,Periwinkle in Cypress Hills,East New York,12,93,https://a0.muscache.com/im/pictures/b748c580-7...,11207,0,1,39.0,12.752202
4996,11906101,Apartment,Private room,"{TV,""Wireless Internet"",""Air conditioning"",Kit...",1,1.0,Real Bed,moderate,True,NYC,...,"Comfy, light filled room in Brooklyn's best hood!",Clinton Hill,7,100,https://a0.muscache.com/im/pictures/c1ad373d-7...,11238,1,1,65.0,9.501860
4997,12917210,House,Entire home/apt,"{TV,""Cable TV"",""Wireless Internet"",""Air condit...",3,1.0,Real Bed,moderate,True,LA,...,Private Studio W/Parking in K- Town near LA Live,Mid-Wilshire,15,100,https://a0.muscache.com/im/pictures/e3b8cad9-1...,90006,0,1,125.0,25.198344
4998,20439522,Apartment,Entire home/apt,"{TV,""Wireless Internet"",""Air conditioning"",Kit...",1,1.0,Real Bed,strict,True,LA,...,Sunset Strip Studio,West Hollywood,15,93,https://a0.muscache.com/im/pictures/110765659/...,90069,0,1,120.0,27.925684
